# Experiment on the encoding and imputing technics for the Nam network, with and without Z ! 

In [1]:
run init.ipynb

Done.


**`NAN` Second set of experiments deciding wheether `using indicator variable is useful`**

In [2]:
"""  Parameters of the experiment """
approach = 'nam'

range_dataset_name = ['blobs', 'circles', 'moons']
range_ratio_of_missing_values = [0., 0.1, 0.2, 0.3]
range_ratio_missing_per_class = [[0,0.25], [0.1, 0.3]]
range_imbalance_ratio = [0.1, 0.25, 0.5]
range_missingness_pattern = [1, 3, 4, 5]

missing_data_handling = 'encoding'#, TODECIDE] #TODO THIS IS WHERE WE SHOULD DO IMUTING BASED ON THE  OTHER EXPERIMENT
imputation_method = 'without'
range_use_missing_indicator_variables = [True, False]

In [3]:
df = create_df()
df


,dataset_name,experiment_number,approach,missing_data_handling,imputation_method,use_missing_indicator_variables,num_samples,imbalance_ratio,missingness_pattern,missingness_mechanism,ratio_of_missing_values,missing_X1,missing_X2,missing_first_quarter,ratio_missing_per_class_0,ratio_missing_per_class_1,auc,Accuracy,F1,MCC,Sensitivity,Specificity,Precision,PPV,NPV,FNR,FDR,FOR,resolution,bandwidth,estimation_time_0,estimation_time_1
0,moons,32,multi_distributions,without,without,None,8000,0.25,5,MNAR,None,True,True,True,0,0.0785,NaN,0.2480,0.2100,-0.3750,0.4070,0.1960,0.1420,0.1420,0.5040,0.5930,0.8580,0.4960,30,0.2,NaN,NaN
1,moons,20,multi_distributions,without,without,None,8000,0.25,1,MCAR,0.3,True,False,None,None,None,NaN,0.3750,0.2080,-0.2380,0.3370,0.3870,0.1500,0.1500,0.6450,0.6630,0.8500,0.3550,30,0.2,NaN,NaN
2,moons,18,multi_distributions,without,without,None,8000,0.25,1,MCAR,0.1,True,False,None,None,None,NaN,0.2890,0.1340,-0.4150,0.2140,0.3150,0.0970,0.0970,0.5370,0.7860,0.9030,0.4630,30,0.2,NaN,NaN
3,moons,27,multi_distributions,without,without,None,8000,0.25,3,MAR,0.2,True,False,True,None,None,NaN,0.2990,0.0980,-0.4310,0.1550,0.3460,0.0720,0.0720,0.5560,0.8450,0.9280,0.4440,30,0.2,NaN,NaN
4,moons,9,multi_distributions,without,without,None,8000,0.10,3,MAR,0,True,False,True,None,None,NaN,0.1760,0.0410,-0.4530,0.1760,0.1760,0.0230,0.0230,0.6600,0.8240,0.9770,0.3400,30,0.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,blobs,13,nam,encoding,without,True,8000,0.10,2,MAR,0.3,True,True,True,None,None,0.9552,0.9711,0.8296,0.8284,0.7100,0.9998,0.9983,0.9983,0.9690,0.2900,0.0017,0.0310,30,0.2,NaN,NaN
108,blobs,5,nam,encoding,without,True,8000,0.10,1,MCAR,0.2,True,False,None,None,None,0.9804,0.9781,0.8759,0.8720,0.7798,0.9999,0.9993,0.9993,0.9763,0.2202,0.0007,0.0237,30,0.2,NaN,NaN
109,blobs,14,nam,encoding,without,True,8000,0.10,3,MAR,0,True,False,True,None,None,1.0000,0.9993,0.9974,0.9971,0.9955,0.9999,0.9993,0.9993,0.9993,0.0045,0.0007,0.0007,30,0.2,NaN,NaN
110,blobs,22,nam,encoding,without,True,8000,0.10,5,MNAR,None,True,True,True,0,0.476875,1.0000,0.9992,0.9967,0.9962,0.9933,1.0000,1.0000,1.0000,0.9992,0.0067,0.0000,0.0008,30,0.2,NaN,NaN


In [ ]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for use_missing_indicator_variables in range_use_missing_indicator_variables:

            for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):


                    if check_experiment_already_done(df, 
                                                      dataset_name=dataset_name, 
                                                      imbalance_ratio=imbalance_ratio, 
                                                      missingness_pattern=missingness_pattern, 
                                                      imputation_method=imputation_method,
                                                      use_missing_indicator_variables=use_missing_indicator_variables,
                                                      ratio_of_missing_values=ratio_of_missing_values, 
                                                      ratio_missing_per_class=ratio_missing_per_class):


                        print("Experiment already done.")
                        continue

                    print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                        missing_data_handling,
                                                                                                                                                          imputation_method, 
                                                                                                                                                          imbalance_ratio,
                                                                                                                                                          missingness_pattern,
                                                                                                                                                          ratio_of_missing_values, 
                                                                                                                                                          ratio_missing_per_class))

                    try:
                        start_time = time()

                        dataset = DatasetGenerator(dataset_name=dataset_name, 
                                                num_samples=num_samples, 
                                                imbalance_ratio=imbalance_ratio, 
                                                missing_data_handling=missing_data_handling,
                                                imputation_method=imputation_method,
                                                verbosity=verbosity)

                        # Creation of the missingness
                        dataset.generate_missing_coordinates(missingness_pattern=missingness_pattern, 
                                                             ratio_of_missing_values=ratio_of_missing_values, 
                                                             ratio_missing_per_class=ratio_missing_per_class)

                        dataset.split_test_train()

                        # Create the experiments
                        exp = Experiments(dataset_name, 
                                          dataset=dataset, 
                                          purpose='classification', 
                                          approach = approach, 
                                          previous_experiment=None, 
                                          save_experiment=save, 
                                          verbosity=verbosity, 
                                          debug=False, 
                                          proportion_train=PROPORTION_TRAIN, 
                                          resolution=RESOLUTION, 
                                          bandwidth=BANDWIDTH,
                                          random_state=RANDOM_STATE)


                        # Estimate the distributions 
                        exp.fit()

                        # Estimate the distributions 
                        exp.predict()

                        # Plot results
                        #exp.plot()


                        end_time = time()

                        hours, rest = divmod(end_time - start_time, 3600)
                        minutes, seconds = divmod(rest, 60)
                        print("Done ({}h {}m and {:.2f}s).\n".format(int(hours), int(minutes), seconds))
                    except:
                        print("/!\. Error!\n")


112
blobs - imputation method: encoding missing_data_handling: without imbalance: 0.1 pattern: 1 ratio_missing:0.0 ratio_per_class: None
Doing experiment 37!
Done (0h 3m and 43.83s).

112
blobs - imputation method: encoding missing_data_handling: without imbalance: 0.1 pattern: 1 ratio_missing:0.1 ratio_per_class: None
Doing experiment 39!
Done (0h 3m and 44.94s).

112
blobs - imputation method: encoding missing_data_handling: without imbalance: 0.1 pattern: 1 ratio_missing:0.2 ratio_per_class: None
Doing experiment 41!
Done (0h 3m and 42.90s).

112
blobs - imputation method: encoding missing_data_handling: without imbalance: 0.1 pattern: 1 ratio_missing:0.3 ratio_per_class: None
Doing experiment 42!
Done (0h 3m and 45.09s).

112
blobs - imputation method: encoding missing_data_handling: without imbalance: 0.1 pattern: 2 ratio_missing:0.0 ratio_per_class: None
Doing experiment 44!
Done (0h 3m and 43.45s).

112
blobs - imputation method: encoding missing_data_handling: without imbalance

In [13]:
dataset = DatasetGenerator(dataset_name=dataset_name, 
                                            num_samples=num_samples, 
                                            imbalance_ratio=imbalance_ratio, 
                                            missing_data_handling=missing_data_handling,
                                            imputation_method=imputation_method,
                                            verbosity=verbosity)

from sklearn import datasets
from sklearn.preprocessing import StandardScaler  
from sklearn.utils import shuffle


In [15]:
self=dataset
self.dataset_name='blobs'
num_samples_gt = 2000

################################
# Generate the positive examples
################################
if self.dataset_name=='moons':

    found=False
    while not found:

        X_all, labels = datasets.make_moons(n_samples=int(2*self.imbalance_ratio*(self.num_samples+num_samples_gt)), noise=.15, random_state=self.random_state)

        idx_out = np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()

        X_all[idx_out], labels[idx_out] = datasets.make_moons(n_samples=len(idx_out), noise=.15)

        if len(np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()) == 0:
            found=True


elif self.dataset_name=='circles':

    found=False
    while not found:
        X_all, labels = datasets.make_circles(n_samples=int(2*self.imbalance_ratio*(self.num_samples+num_samples_gt)), factor=.5, noise=.15, random_state=self.random_state)

        idx_out = np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()

        X_all[idx_out], labels[idx_out] = datasets.make_circles(n_samples=len(idx_out), factor=.5, noise=.15, random_state=self.random_state)

        if len(np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()) == 0:
            found=True


elif self.dataset_name=='blobs':

    found=False
    while not found:

        X_all, labels = datasets.make_blobs(n_samples=int(2*self.imbalance_ratio*(self.num_samples+num_samples_gt)), centers=[[-1, 0],[1, 0]], cluster_std=.3, random_state=self.random_state)

        idx_out = np.argwhere( (X_all[:,0]>2.4) | (X_all[:,0] < -2.4) | (X_all[:,1]>2.4) | (X_all[:,1] < -2.4) ).squeeze()

        X_all[idx_out], labels[idx_out] = datasets.make_blobs(n_samples=len(idx_out), centers=[[-1, 0],[1, 0]],
                                                                cluster_std=.05)

        if len(np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()) == 0:
            found=True

else:
    raise ValueError("Please use 'moons', 'circles', or 'blobs' datasets.")             

# normalize dataset for easier parameter selection
X_all = StandardScaler().fit_transform(X_all)

# Select the positive examples
X_all = X_all[np.argwhere(labels==1).squeeze()]

# Separate ground truth and training data
X_pos = X_all[:int(self.num_samples*self.imbalance_ratio),:]
#Xgt_pos = X_all[int(num_samples*imbalance_ratio):,:]
labels_pos = 1*np.ones((X_pos.shape[0], 1))

#labelsgt_pos  = 1*np.ones((Xgt_pos.shape[0], 1))

################################
# Generate the negative examples
################################
if self.dataset_name=='moons':

    found=False
    while not found:

        X_all, labels = datasets.make_moons(n_samples=int(2*(1-self.imbalance_ratio)*(self.num_samples+num_samples_gt)), noise=.15, random_state=self.random_state)
        idx_out = np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()

        X_all[idx_out], labels[idx_out] = datasets.make_moons(n_samples=len(idx_out), noise=.15)

        if len(np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()) == 0:
            found=True

elif self.dataset_name=='circles':

    found=False
    while not found:

        X_all, labels = datasets.make_circles(n_samples=int(2*(1-self.imbalance_ratio)*(self.num_samples+num_samples_gt)), factor=.5, noise=.15, random_state=self.random_state)

        idx_out = np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()

        X_all[idx_out], labels[idx_out] = datasets.make_circles(n_samples=len(idx_out), factor=.5, noise=.15, random_state=self.random_state)

        if len(np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()) == 0:
            found=True



elif self.dataset_name=='blobs':

    found=False
    while not found:

        X_all, labels = datasets.make_blobs(n_samples=int(2*(1-self.imbalance_ratio)*(self.num_samples+num_samples_gt)), centers=[[-1, 0],[1, 0]], cluster_std=.3, random_state=self.random_state)

        idx_out = np.argwhere( (X_all[:,0]>2.4) | (X_all[:,0] < -2.4) | (X_all[:,1]>2.4) | (X_all[:,1] < -2.4) ).squeeze()

        X_all[idx_out], labels[idx_out] = datasets.make_blobs(n_samples=len(idx_out), centers=[[-1, 0],[1, 0]],
                                                                cluster_std=.05, random_state=self.random_state)


else:
    raise ValueError("Please use 'moons', 'circles', or 'blobs' datasets.")               

# normalize dataset for easier parameter selection
X_all = StandardScaler().fit_transform(X_all)

# Select the negative examples
X_all = X_all[np.argwhere(labels==0).squeeze()]

# Separate ground truth and training data
X_neg = X_all[:int(self.num_samples*(1-self.imbalance_ratio)),:] 
#Xgt_neg = X_all[int(num_samples*(1-imbalance_ratio)):,:]
labels_neg = np.zeros((X_neg.shape[0], 1))
#labelsgt_neg = np.zeros((Xgt_neg.shape[0], 1))

# Combine the positive and negative samples
X, y = np.concatenate([X_neg, X_pos], axis=0), np.concatenate([labels_neg, labels_pos], axis=0)
#X_gt, y_gt = np.concatenate([Xgt_neg, Xgt_pos], axis=0), np.concatenate([labelsgt_neg, labelsgt_pos], axis=0)

# Shuffle the data 
X_raw, y = shuffle(X, y, random_state=self.random_state)
#self.X_gt, self.y_gt = shuffle(X_gt, y_gt, random_state=random_state)

X_raw, y

(array([[ 1.60952648, -0.72029916],
        [ 1.03056358,  1.6777475 ],
        [-1.31510581,  1.1735564 ],
        ...,
        [-0.38177019,  0.98798329],
        [ 0.64550682, -2.13086448],
        [ 1.59439813, -1.65256827]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]))

In [17]:

found=False
while not found:
    print('y')

    X_all, labels = datasets.make_blobs(n_samples=int(2*self.imbalance_ratio*(self.num_samples+num_samples_gt)), centers=[[-1, 0],[1, 0]], cluster_std=.3, random_state=self.random_state)

    idx_out = np.argwhere( (X_all[:,0]>2.4) | (X_all[:,0] < -2.4) | (X_all[:,1]>2.4) | (X_all[:,1] < -2.4) ).squeeze()

    X_all[idx_out], labels[idx_out] = datasets.make_blobs(n_samples=len(idx_out), centers=[[-1, 0],[1, 0]],
                                                            cluster_std=.05)

    if len(np.argwhere( (X_all[:,0]>2.49) | (X_all[:,0] < -2.49) | (X_all[:,1]>2.49) | (X_all[:,1] < -2.49) ).squeeze()) == 0:
        found=True

y
